In [2]:
pip install -U scikit-fuzzy

     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-cp36-none-any.whl size=894070 sha256=eaa2c30c54dd4dc120a35cd6224640405e0165a48fdb2200271b6e49d5d90deb
  Stored in directory: /root/.cache/pip/wheels/b9/4e/77/da79b16f64ef1738d95486e2731eea09d73e90a72465096600
Successfully built scikit-fuzzy


In [3]:
pip install twython

In [4]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [1]:
import pandas as pd
import re
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import io

start = time.time()

In [12]:
from google.colab import files
upload = files.upload() 
traindata = pd.read_csv(io.StringIO(upload['SemEval2017-task4-dev.subtask-A.csv'].decode('ISO-8859-1')))

Saving SemEval2017-task4-dev.subtask-A.csv to SemEval2017-task4-dev.subtask-A.csv


In [ ]:
# Used for two class classification
# all_negative_tweets=[]
# all_positive_tweets=[]

# for j in range(len(traindata.tweet_text)):
#   if(traindata.sentiment[j]=="Negative"):
#     all_negative_tweets.append(traindata.tweet_text[j])
#   else:
#     all_positive_tweets.append(traindata.tweet_text[j])

In [13]:
doc=[]
sentidoc=[]
k=0
for j in range(len(traindata.tweet_text)):
  if traindata.sentiment[j]=='irrelevant':
    k+=1
    continue
  else:
    doc.append(traindata.tweet_text[j])
    sentidoc.append(traindata.sentiment[j])
print(k)
print(len(doc))
len(sentidoc)

0
20632


20632

Preprocessing

In [14]:
tweets=[]
senti=[]


for j in range(len(doc)):
    str1=doc[j]
    str2=str1.lower()
    tweets.append(str2)   # converted into lower case
    senti.append(sentidoc[j])
print(len(senti))
len(tweets)

20632


20632

In [22]:
print(senti[66])

Neutral


In [15]:
def preprocessing(phrase):   # text pre-processing 
        # specific
        phrase = re.sub(r'\W', ' ', phrase)  # Remove all the special characters
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)
        phrase = re.sub(r"@", "" , phrase)         # removal of @
        phrase =  re.sub(r"http\S+", "", phrase)   # removal of URLs
        phrase = re.sub(r"#", "", phrase)          # hashtag processing

        phrase= re.sub(r'\s+[a-zA-Z]\s+', ' ', phrase)  # remove all single characters
   
        phrase = re.sub(r'\^[a-zA-Z]\s+', ' ', phrase)          # Remove single characters from the start
    # Substituting multiple spaces with single space
        phrase = re.sub(r'\s+', ' ', phrase, flags=re.I)

    # Removing prefixed 'b'
        phrase = re.sub(r'^b\s+', '', phrase)
    
        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

In [16]:
for k in range(len(doc)):
    tweets[k]=preprocessing(tweets[k])

In [27]:
print(tweets[0])

 aapl the 10 best steve jobs emails ever http co 82g1kl94tx


Generate universe variables
   * pos and neg on subjective ranges [0, 1]
   * op has a range of [0, 10] in units of percentage points

In [17]:
x_p = np.arange(0, 1, 0.1)
x_n = np.arange(0, 1, 0.1)
x_op = np.arange(0, 10, 1)

# Generate fuzzy membership functions
p_lo = fuzz.trimf(x_p, [0, 0, 0.5])
p_md = fuzz.trimf(x_p, [0, 0.5, 1])
p_hi = fuzz.trimf(x_p, [0.5, 1, 1])
n_lo = fuzz.trimf(x_n, [0, 0, 0.5])
n_md = fuzz.trimf(x_n, [0, 0.5, 1])
n_hi = fuzz.trimf(x_n, [0.5, 1, 1])
op_Neg = fuzz.trimf(x_op, [0, 0, 5])  # Scale : Neg Neu Pos
op_Neu = fuzz.trimf(x_op, [0, 5, 10])
op_Pos = fuzz.trimf(x_op, [5, 10, 10])

In [18]:
sid = SentimentIntensityAnalyzer()
sentiment=[]
sentiment_doc=[]
Out=[]
for j in range(len(doc)):
    sentiment_doc.append(senti[j])
    ss = sid.polarity_scores(tweets[j])
    posscore=ss['pos']
    negscore=ss['neg']
    neuscore=ss['neu']
    compoundscore=ss['compound']
   
    print(str(j+1)+" {:-<65} {}".format(tweets[j], str(ss))) 
    
    print("\nPositive Score for each  tweet :")    
    if (posscore==1):
        posscore=0.9 
    else:
        posscore=round(posscore,1)
    print(posscore)

    print("\nNegative Score for each  tweet :")
    if (negscore==1):
        negscore=0.9
    else:
        negscore=round(negscore,1)
    print(negscore)

# We need the activation of our fuzzy membership functions at these values.
    p_level_lo = fuzz.interp_membership(x_p, p_lo, posscore)
    p_level_md = fuzz.interp_membership(x_p, p_md, posscore)
    p_level_hi = fuzz.interp_membership(x_p, p_hi, posscore)
    
    n_level_lo = fuzz.interp_membership(x_n, n_lo, negscore)
    n_level_md = fuzz.interp_membership(x_n, n_md, negscore)
    n_level_hi = fuzz.interp_membership(x_n, n_hi, negscore)
    
    # Now we take our rules and apply them. Rule 1 concerns bad food OR nice.
    # The OR operator means we take the maximum of these two.
    active_rule1 = np.fmin(p_level_lo, n_level_lo)
    active_rule2 = np.fmin(p_level_md, n_level_lo)
    active_rule3 = np.fmin(p_level_hi, n_level_lo)
    active_rule4 = np.fmin(p_level_lo, n_level_md)
    active_rule5 = np.fmin(p_level_md, n_level_md)
    active_rule6 = np.fmin(p_level_hi, n_level_md)
    active_rule7 = np.fmin(p_level_lo, n_level_hi)
    active_rule8 = np.fmin(p_level_md, n_level_hi)
    active_rule9 = np.fmin(p_level_hi, n_level_hi)
    
    # Now we apply this by clipping the top off the corresponding output
    # membership function with `np.fmin`
    
    n1=np.fmax(active_rule4,active_rule7)
    n2=np.fmax(n1,active_rule8)     
    op_activation_lo = np.fmin(n2,op_Neg)
    
    neu1=np.fmax(active_rule1,active_rule5)
    neu2=np.fmax(neu1,active_rule9)     
    op_activation_md = np.fmin(neu2,op_Neu)
    
    p1=np.fmax(active_rule2,active_rule3)
    p2=np.fmax(p1,active_rule6)   
    op_activation_hi = np.fmin(p2,op_Pos)
    
    op0 = np.zeros_like(x_op)
    
    # Aggregate all three output membership functions together
    aggregated = np.fmax(op_activation_lo,
                         np.fmax(op_activation_md, op_activation_hi))
    
    # Calculate defuzzified result
    op = fuzz.defuzz(x_op, aggregated, 'centroid')
    output=round(op,2)
    print("OUTPUt",output)
    Out.append(output)
   
    op_activation = fuzz.interp_membership(x_op, aggregated, op)  # for plot
    if 0<(output)<3.33:    # R
        print("\nOutput after Defuzzification: Negative")
        sentiment.append("Negative")

    elif 3.33<(output)<6.66:
        print("\nOutput after Defuzzification: Neutral")
        sentiment.append("Neutral")
        
    elif 6.66<(output)<10:
        print("\nOutput after Defuzzification: Positive")
        sentiment.append("Positive")
        
    print("Doc sentiment: " +str(senti[j])+"\n")
    print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")

Streaming output truncated to the last 5000 lines.
0.0
OUTPUt 4.91

Output after Defuzzification: Neutral
Doc sentiment: Neutral

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
20249  patrickcoffee nickchilds you may also be interested in this story about kim kardashian _______ {'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'compound': 0.4019}

Positive Score for each  tweet :
0.2

Negative Score for each  tweet :
0.0
OUTPUt 4.98

Output after Defuzzification: Neutral
Doc sentiment: Neutral

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
20250  guardian chris evans has been successful in everything he down amp great viewing figures but young generation may not forgive clarkson axe  {'neg': 0.083, 'neu': 0.676, 'pos': 0.24, 'compound': 0.4893}

Positive Score for each  tweet :
0.2

Negative Score for each  tweet :
0.1
OUTPUt 4.88

Output after Defuzzification: Neutral
Doc sentiment: Positive

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [19]:
print(len(sentiment))
print(len(sentiment_doc))
len(Out)

20632
20632


20632

In [ ]:
# for i in range(len(sentiment_doc)):
#   if sentiment_doc[i]=='Neutral / author is just sharing information':
#     sentiment_doc[i]='Neutral'

In [20]:
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score
y_true = sentiment_doc
y_pred = sentiment

f1ma=f1_score(y_true, y_pred, average='macro')
f1mi=f1_score(y_true, y_pred, average='micro')
pma=precision_score(y_true, y_pred, average='macro')
pmi=precision_score(y_true, y_pred, average='micro')
rma=recall_score(y_true, y_pred, average='macro')
rmi=recall_score(y_true, y_pred, average='micro')
acc=accuracy_score(y_true, y_pred)
print('f1ma',f1ma)
print('f1mi',f1mi)
print('pma',pma)
print('pmi',pmi)
print('rma',rma)
print('rmi',rmi)
print('acc',acc)

f1ma 0.27698880293777656
f1mi 0.516285381930981
pma 0.7120024005491117
pmi 0.516285381930981
rma 0.35888267680559327
rmi 0.516285381930981
acc 0.516285381930981
